<a href="https://colab.research.google.com/github/riphunter7001x/MultiModal_RAG/blob/main/Chat_With_Multiple_Doc(pdfs%2C_docs%2C_txt%2C_pptx)_using_AstraDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain
!pip install unstructured
!pip install openai
!pip install Cython
!pip install tiktoken

In [2]:
!pip install --upgrade langchain-astradb

In [3]:
!pip install langchain langchain-openai datasets pypdf

In [4]:
!pip install pdf2image

In [5]:
!pip install pdfminer.six

In [6]:
!pip install unstructured[pdf]

In [8]:
%pip install --upgrade --quiet  sentence_transformers langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 37.1 MB/s eta 0:00:00


In [9]:
import os
from getpass import getpass

from datasets import (
    load_dataset,
)
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.indexes import VectorstoreIndexCreator

In [11]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
embedding = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

# Using Unstructured for loading Multiple Pdfs

In [ ]:
root_dir="/content/"

In [ ]:
pdf_folder_path = f'{root_dir}/docs/'

In [ ]:
os.listdir(pdf_folder_path)

In [ ]:
# location of the pdf file/files.
loaders = [UnstructuredPDFLoader(os.path.join(pdf_folder_path, fn)) for fn in os.listdir(pdf_folder_path)]

In [ ]:
loaders

In [ ]:
index = VectorstoreIndexCreator().from_loaders(loaders)

In [ ]:
index.query('What is the tokenization in RAG?')

In [ ]:
index.query_with_sources('What is the tokenization in RAG?')

# Pypdf loader with Multiple Pdfs.

In [ ]:
from langchain_astradb import AstraDBVectorStore

In [ ]:
from langchain_astradb import AstraDBVectorStore
ASTRA_DB_API_ENDPOINT="https://d2357619-8f04-4cfd-bc3a-16e410893ba3-us-east-2.apps.astra.datastax.com"
ASTRA_DB_APPLICATION_TOKEN="AstraCS:hTmlZSqmAOUHSWZaeNqzEDOR:1128826e960e49c2508b3014ae7fa40e6b5d0490d8565702a30b4ea338083a4a"
ASTRA_DB_KEYSPACE="default_keyspace"

In [ ]:
root_dir="/content/"
pdf_folder_path = f'{root_dir}/data/'
pdfs=os.listdir(pdf_folder_path)

In [ ]:
pdfs

['MachineTranslationwithAttention.pdf',
 'Retrieval-Augmented-Generation-for-NLP.pdf']

In [ ]:
data=PyPDFLoader("/content/data/MachineTranslationwithAttention.pdf")

In [ ]:
data

In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=64)

In [ ]:
data.load_and_split(text_splitter=splitter)

[Document(page_content='Retrieval-Augmented Generation for\nKnowledge-Intensive NLP Tasks\nPatrick Lewis†‡, Ethan Perez⋆,\nAleksandra Piktus†, Fabio Petroni†, Vladimir Karpukhin†, Naman Goyal†, Heinrich Küttler†,\nMike Lewis†, Wen-tau Yih†, Tim Rocktäschel†‡, Sebastian Riedel†‡, Douwe Kiela†\n†Facebook AI Research;‡University College London;⋆New York University;\nplewis@fb.com\nAbstract\nLarge pre-trained language models have been shown to store factual knowledge', metadata={'source': '/content/data/Retrieval-Augmented-Generation-for-NLP.pdf', 'page': 0}),
 Document(page_content='in their parameters, and achieve state-of-the-art results when ﬁne-tuned on down-\nstream NLP tasks. However, their ability to access and precisely manipulate knowl-\nedge is still limited, and hence on knowledge-intensive tasks, their performance\nlags behind task-speciﬁc architectures. Additionally, providing provenance for their\ndecisions and updating their world knowledge remain open research problems. Pr

In [ ]:
docs=[]
for pdf in pdfs:
  data=PyPDFLoader(f"/content/data/{pdf}")
  docs.append(data)

In [ ]:

docs_from_pdf = docs.load_and_split(text_splitter=splitter)

AttributeError: 'list' object has no attribute 'load_and_split'

In [ ]:
print(f"Documents from PDF: {len(docs_from_pdf)}.")
inserted_ids_from_pdf = vstore.add_documents(docs_from_pdf)
print(f"Inserted {len(inserted_ids_from_pdf)} documents.")

In [ ]:
vstore = AstraDBVectorStore(
    embedding=embedding,
    collection_name="astra_vector_demo",
    api_endpoint=ASTRA_DB_API_ENDPOINT,
    token=ASTRA_DB_APPLICATION_TOKEN,
    namespace=ASTRA_DB_KEYSPACE,
)

In [ ]:
retriever = vstore.as_retriever(search_kwargs={"k": 3})

In [ ]:
prompt_template = """
You are a philosopher that draws inspiration from great thinkers of the past
to craft well-thought answers to user questions. Use the provided context as the basis
for your answers and do not make up new reasoning paths - just mix-and-match what you are given.
Your answers must be concise and to the point, and refrain from answering about other topics than philosophy.

CONTEXT:
{context}

QUESTION: {question}

YOUR ANSWER:"""

In [ ]:
prompt_template = ChatPromptTemplate.from_template(prompt_template)

In [ ]:
llm = ChatOpenAI()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | philo_prompt
    | llm
    | StrOutputParser()
)

In [ ]:
chain.invoke("How does Russel elaborate on Peirce's idea of the security blanket?")

# Directory loders(Chat With Multiple Doc)

In [ ]:
!rm -rf "/content/docs/.ipynb_checkpoints"

In [ ]:
%pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 19.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [12]:
!pip install unstructured


In [13]:
!pip install "unstructured[pdf]"

In [14]:
!sudo apt-get update

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [51.5 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,386 kB]
Get:13 http://archive.ubuntu.com/ubunt

In [15]:
!sudo apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 48 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.4 [186 kB]
Fetched 186 kB in 0s (1,143 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package poppler-utils.
(Reading database ... 12191

In [16]:
!sudo apt-get install libleptonica-dev tesseract-ocr libtesseract-dev python3-pil tesseract-ocr-eng tesseract-ocr-script-latn


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libarchive-dev libarchive13 libimagequant0 libraqm0 mailcap mime-support
  python3-olefile tesseract-ocr-osd
Suggested packages:
  lrzip python-pil-doc
The following NEW packages will be installed:
  libarchive-dev libimagequant0 libleptonica-dev libraqm0 libtesseract-dev
  mailcap mime-support python3-olefile python3-pil tesseract-ocr
  tesseract-ocr-eng tesseract-ocr-osd tesseract-ocr-script-latn
The following packages will be upgraded:
  libarchive13
1 upgraded, 13 newly installed, 0 to remove and 47 not upgraded.
Need to get 40.3 MB of archives.
After this operation, 123 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libarchive13 amd64 3.6.0-1ubuntu1.1 [369 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libarchive-dev amd64 3.6.0-1ubuntu1.1 [582 kB]
Get:

In [17]:
!pip install unstructured-pytesseract
!pip install tesseract-ocr

  Preparing metadata (setup.py) ... done
  Created wheel for tesseract-ocr: filename=tesseract_ocr-0.0.1-cp310-cp310-linux_x86_64.whl size=169755 sha256=b4460aac3aa42cb0472449d4d53ef93c1b24582f20b1ac4404aa88528b92c708
  Stored in directory: /root/.cache/pip/wheels/bb/fd/f3/5c231ecbbb80a1fe33204ff3021d99b54ef6daf6f8099311b8
Successfully built tesseract-ocr


In [18]:
!pip install "unstructured[pptx]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 16.5 MB/s eta 0:00:00


In [19]:
!pip install langchain_astradb

In [20]:
!pip install langchain langchain-openai datasets pypdf

In [21]:
!pip install "unstructured[docx]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 5.3 MB/s eta 0:00:00


In [52]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [53]:
from langchain_community.document_loaders import DirectoryLoader


In [54]:
loader = DirectoryLoader('/content/docs')

In [55]:
splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=64)

In [56]:
docs = loader.load_and_split(text_splitter=splitter)

In [61]:
docs

[Document(page_content='BTECH FINAL YEAR PROJECT SYNOPSIS A Synopsis Submitted in Partial Fulfillment of the Requirements for the Degree of BACHELOR OF TECHNOLOGY in Department of Artificial Intelligence\n\nGuided by : Prof. Shubhangi Ingale\n\nProject Topic / Title : Movie Review Sentiment Analysis on IMDB Data by Vishal Kshirsagar AIB14 Aditya Varpe AIA07 Shivam Kharat AIA62 Vivek Vaidya  BCOB121', metadata={'source': '/content/docs/syno.txt'}),
 Document(page_content='G H Raisoni College of Engineering and Management Wagholi, Pune Department of Artificial Intelligence (An Autonomous Institute Affiliated to SPPU, Pune)\n\nMarch, 2024\n\nSynopsis\n\n1. Introduction', metadata={'source': '/content/docs/syno.txt'}),
 Document(page_content='In an age dominated by digital discourse and the democratization of opinions, the film industry stands at the forefront of this cultural dialogue. The democratization of movie reviews on platforms like IMDB has transformed the audience from mere spect

In [58]:
import os
from langchain_core.documents import Document
from langchain_community.document_loaders import PyPDFLoader

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings



In [59]:
import os
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [60]:
!pip install langchain_google_genai sentence_transformers

In [37]:
from langchain_google_genai import ChatGoogleGenerativeAI
def load_model(model_name):
  if model_name=="gemini-pro":
    llm = ChatGoogleGenerativeAI(model="gemini-pro")
  else:
    llm=ChatGoogleGenerativeAI(model="gemini-pro-vision")

  return llm


llm=load_model("gemini-pro")

In [ ]:
# from sentence_transformers import SentenceTransformer
# embedding = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [34]:
from langchain_astradb import AstraDBVectorStore
from langchain.indexes import VectorstoreIndexCreator

In [35]:
ASTRA_DB_API_ENDPOINT="https://aa7cfa87-c077-41fd-98b2-c2ccaceff222-us-east-2.apps.astra.datastax.com"
ASTRA_DB_APPLICATION_TOKEN="AstraCS:zSbUrMqcmgsTJmbZPoHfYhjU:0a6c3bd57eaa93181111477fe6f08c1fcde65660a2c00ba250057a6b0182aa14"
ASTRA_DB_KEYSPACE="default_keyspace"

In [68]:
vstore = AstraDBVectorStore(
    embedding=embedding,
    collection_name="multidoc_vector",
    api_endpoint=ASTRA_DB_API_ENDPOINT,
    token=ASTRA_DB_APPLICATION_TOKEN,
    namespace=ASTRA_DB_KEYSPACE,
)

In [69]:
inserted_ids = vstore.add_documents(docs)

In [70]:
print(f"\nInserted {len(inserted_ids)} documents.")


Inserted 204 documents.


In [72]:
results = vstore.similarity_search("title of project? ", k=3)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* BTECH FINAL YEAR PROJECT SYNOPSIS A Synopsis Submitted in Partial Fulfillment of the Requirements for the Degree of BACHELOR OF TECHNOLOGY in Department of Artificial Intelligence

Guided by : Prof. Shubhangi Ingale

Project Topic / Title : Movie Review Sentiment Analysis on IMDB Data by Vishal Kshirsagar AIB14 Aditya Varpe AIA07 Shivam Kharat AIA62 Vivek Vaidya  BCOB121 [{'source': '/content/docs/syno.txt'}]
* BTECH FINAL YEAR PROJECT SYNOPSIS A Synopsis Submitted in Partial Fulfillment of the Requirements for the Degree of BACHELOR OF TECHNOLOGY in Department of Artificial Intelligence

Guided by : Prof. Shubhangi Ingale

Project Topic / Title : Movie Review Sentiment Analysis on IMDB Data by Vishal Kshirsagar AIB14 Aditya Varpe AIA07 Shivam Kharat AIA62 Vivek Vaidya  BCOB121 [{'source': '/content/docs/syno.txt'}]
* BTECH FINAL YEAR PROJECT SYNOPSIS A Synopsis Submitted in Partial Fulfillment of the Requirements for the Degree of BACHELOR OF TECHNOLOGY in Department of Artificial I

In [73]:
prompt_template = """
You are an AI philosopher drawing insights from the of "rag," "llama3," and "genai."
Craft thoughtful answers based on this roadmap, mixing and matching existing paths.
Your responses should be concise and strictly related to the provided context.

ROADMAP CONTEXT:
{context}

QUESTION: {question}

YOUR ANSWER:"""

In [74]:
prompt_template = ChatPromptTemplate.from_template(prompt_template)

In [75]:
retriever = vstore.as_retriever(search_kwargs={"k": 3})

In [76]:
retriever

VectorStoreRetriever(tags=['AstraDBVectorStore', 'HuggingFaceBgeEmbeddings'], vectorstore=<langchain_astradb.vectorstores.AstraDBVectorStore object at 0x78a6a501e6b0>, search_kwargs={'k': 3})

In [77]:
llm = ChatOpenAI()

ValidationError: 1 validation error for ChatOpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

In [78]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [81]:
chain.invoke("can you tell title of project ???")

'Movie Review Sentiment Analysis on IMDB Data'

'Llama (Large Language Model Meta AI) is a family of autoregressive large language models released by Meta AI. The latest version is Llama 3, which was released in April 2024. Llama models have shown superior performance compared to other large language models, such as GPT-3 and PaLM. These models range in parameter sizes from 7B to 70B and have been made accessible for both academic and commercial use.'